In [4]:
!pip install llm


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [137]:
from openai import OpenAI
import os
import ast
import re
import json

import pandas as pd

In [80]:
#data_importance = pd.read_csv("./plus0-ecosystem_mean_feature_importance_values-signed.tsv", header=0, sep="\t")
data_importance_orig = pd.read_csv("/Users/marcin/Documents/KBase/KE/MGnify/paper/pseudoabundance_vs_importance/ecosystem-feature-shap-pabund-corr.tsv", header=0, index_col=0, sep="\t")
data_importance_orig

,GO:0000502 proteasome complex,GO:0001510 RNA methylation,GO:0003950 NAD+ ADP-ribosyltransferase activity,GO:0003968 RNA-dependent RNA polymerase activity,GO:0004037 allantoicase activity,"GO:0004114 3',5'-cyclic-nucleotide phosphodiesterase activity",GO:0004122 cystathionine beta-synthase activity,GO:0004144 diacylglycerol O-acyltransferase activity,GO:0004352 glutamate dehydrogenase (NAD+) activity,GO:0004521 RNA endonuclease activity,...,sk__Bacteria;k__;p__Proteobacteria;c__Epsilonproteobacteria;o__Campylobacterales;f__Campylobacteraceae;g__Arcobacter,sk__Bacteria;k__;p__Spirochaetes;c__Spirochaetia;o__Spirochaetales;f__Spirochaetaceae;g__Treponema;s__Treponema_porcinum,sk__Bacteria;k__;p__Tenericutes;c__Mollicutes,sk__Bacteria;k__;p__Verrucomicrobia;c__Spartobacteria;o__Chthoniobacterales;f__Chthoniobacteraceae;g__Candidatus_Udaeobacter,sk__Bacteria;k__;p__Verrucomicrobia;c__Verrucomicrobiae;o__Verrucomicrobiales;f__Akkermansiaceae;g__Akkermansia,sk__Eukaryota;k__;p__;c__Dinophyceae;o__Suessiales;f__Symbiodiniaceae;g__Symbiodinium,sk__Eukaryota;k__Fungi;p__Ascomycota;c__Eurotiomycetes;o__Eurotiales;f__Aspergillaceae,sk__Eukaryota;k__Fungi;p__Basidiomycota;c__Ustilaginomycetes;o__Ustilaginales;f__Ustilaginaceae,sk__Eukaryota;k__Metazoa;p__Annelida;c__Polychaeta;o__Terebellida,sk__Eukaryota;k__Metazoa;p__Arthropoda;c__Ostracoda;o__Halocyprida
Ecosystem,,,,,,,,,,,,,,,,,,,,,
root:Engineered:Bioreactor,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
root:Engineered:Bioremediation:Terephthalate:Wastewater,0.000000,0.000000,0.000000,0.000000,0.542480,0.579543,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
root:Engineered:Built environment,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.865282,0.000000,0.000000,0.000000
root:Engineered:Food production,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
root:Engineered:Food production:Dairy products,0.000000,0.929234,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
root:Engineered:Food production:Fermented beverages,0.000000,0.000000,0.000000,0.000000,0.000000,0.759914,0.939782,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
root:Engineered:Solid waste:Composting,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.680234,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
root:Engineered:Wastewater,0.000000,0.000000,0.000000,0.722396,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.731658,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
root:Engineered:Wastewater:Activated Sludge,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [7]:
prompt_templates = {
    "summarize_text": "Summarize the following text:\n\n{text}\n",
    "generate_ideas": "Generate some ideas about the following topic:\n\n{topic}\n",
    "question_answer": "Answer the following question:\n\n{question}\n"
}


In [16]:

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)


In [111]:


def query_openai(model, role, prompt):

    chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": role,
            "content": prompt,
        }
    ],
    model=model,#"gpt-3.5-turbo",
    )
    return chat_completion



In [112]:
# Example to answer a question
question = "Which environments are the following biological features likely to be found together?"
question = question + ""
answer = query_openai("gpt-4o", "user", question)
print("Answer:", answer)

Answer: ChatCompletion(id='chatcmpl-9WdoLL7ACrDarX5WoZZc3iM0wwEb5', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="To answer your question accurately, it would be helpful to know the specific biological features you're referring to. Generally speaking, different combinations of traits and adaptations are typical of specific environments. For instance:\n\n1. **Desert Environments**:\n    - Cacti with thick, fleshy bodies and spines.\n    - Animals like camels with water storage capacities and tolerance for high temperatures.\n    - Reptiles such as lizards that are specially adapted to conserve water and regulate their body heat.\n\n2. **Rainforest Environments**:\n    - Large, broad-leaved plants and trees with canopies.\n    - Diverse species of insects, birds, and mammals like monkeys and sloths.\n    - Epiphytic plants like orchids and ferns that grow on other plants.\n\n3. **Marine Environments**:\n    - Coral reefs with diverse

In [113]:
answerdict = answer.to_dict()
content = answerdict['choices'][0]['message']['content']
print("Answer:", content)

Answer: To answer your question accurately, it would be helpful to know the specific biological features you're referring to. Generally speaking, different combinations of traits and adaptations are typical of specific environments. For instance:

1. **Desert Environments**:
    - Cacti with thick, fleshy bodies and spines.
    - Animals like camels with water storage capacities and tolerance for high temperatures.
    - Reptiles such as lizards that are specially adapted to conserve water and regulate their body heat.

2. **Rainforest Environments**:
    - Large, broad-leaved plants and trees with canopies.
    - Diverse species of insects, birds, and mammals like monkeys and sloths.
    - Epiphytic plants like orchids and ferns that grow on other plants.

3. **Marine Environments**:
    - Coral reefs with diverse fish species.
    - Marine mammals such as dolphins and whales.
    - Seaweed and other marine algae.

4. **Arctic/Antarctic Environments**:
    - Polar bears and penguins.


In [114]:
eco_feat_df = pd.read_csv("/Users/marcin/Documents/KBase/KE/KE/notebooks/characteristic_feature_GO_IPR_taxa.tsv",sep="\t",header=0, index_col=0)
eco_feat_df

,Ecosystem,Features_and_Taxa
0,root:Engineered:Bioreactor,"['GO:0008743', 'GO:0018551', 'GO:0031460', 'IP..."
1,root:Engineered:Bioremediation:Terephthalate:W...,"['GO:0004037', 'GO:0004114', 'GO:0008743', 'GO..."
2,root:Engineered:Built environment,"['IPR009413', 'IPR010960', 'IPR021822', 'IPR02..."
3,root:Engineered:Food production,"['GO:0004638', 'GO:0004984', 'IPR000477', 'IPR..."
4,root:Engineered:Food production:Dairy products,"['GO:0001510', 'GO:0004638', 'GO:0008740', 'GO..."
5,root:Engineered:Food production:Fermented beve...,"['GO:0004114', 'GO:0004122', 'GO:0005871', 'GO..."
6,root:Engineered:Solid waste:Composting,"['GO:0004122', 'GO:0004638', 'GO:0006928', 'GO..."
7,root:Engineered:Wastewater,"['GO:0003968', 'GO:0008901', 'GO:0015858', 'GO..."
8,root:Engineered:Wastewater:Activated Sludge,"['GO:0006928', 'GO:0008901', 'GO:0015444', 'GO..."
9,root:Engineered:Wastewater:Water and sludge,"['GO:0004521', 'GO:0017148', 'GO:0019068', 'GO..."


In [157]:
def parse_features(features):
    features_dict = {}
    for feature in features:
        if feature.startswith('GO:') or feature.startswith('IPR'):
            parts = feature.split(' ', 1)
            if len(parts) == 2:
                feature_id, label = parts
                features_dict[feature_id] = feature_id + " - "+label#feature_id +" - "+
        elif feature.startswith('sk__'):
            features_dict[feature] = " - "+feature
    return features_dict
    
parsed_features = parse_features(data_importance_orig.columns)
parsed_features

{'GO:0000502': 'GO:0000502 - proteasome complex',
 'GO:0001510': 'GO:0001510 - RNA methylation',
 'GO:0003950': 'GO:0003950 - NAD+ ADP-ribosyltransferase activity',
 'GO:0003968': 'GO:0003968 - RNA-dependent RNA polymerase activity',
 'GO:0004037': 'GO:0004037 - allantoicase activity',
 'GO:0004114': "GO:0004114 - 3',5'-cyclic-nucleotide phosphodiesterase activity",
 'GO:0004122': 'GO:0004122 - cystathionine beta-synthase activity',
 'GO:0004144': 'GO:0004144 - diacylglycerol O-acyltransferase activity',
 'GO:0004352': 'GO:0004352 - glutamate dehydrogenase (NAD+) activity',
 'GO:0004521': 'GO:0004521 - RNA endonuclease activity',
 'GO:0004638': 'GO:0004638 - phosphoribosylaminoimidazole carboxylase activity',
 'GO:0004984': 'GO:0004984 - olfactory receptor activity',
 'GO:0005471': 'GO:0005471 - ATP:ADP antiporter activity',
 'GO:0005871': 'GO:0005871 - kinesin complex',
 'GO:0006011': 'GO:0006011 - UDP-glucose metabolic process',
 'GO:0006080': 'GO:0006080 - substituted mannan metabol

In [170]:
prompt_template_orig = """
***Begin Environmental Ecology and Microbiology Prompt
You are a microbiology, ecology, and environmental expert tasked with searching for and 
explaining relationships between different mostly microbial metagenomic features (with 
some eukaryotic features also present) observed to be important 
in a specific environment and their relationship to properties of that environment.

In our conducted <Ecosystem> ecosystem experiment that we are working now to interpret, 
we can observe important Gene Ontology GO terms, InterPro IPR sequence domains, and 
sk__ NCBI Taxonomy features represented as standard identifiers and their 
descriptions.

For each feature or feature subset, the goal is to identify confident relationships 
between specific feature or feature subsets with specific properties of that 
environment. Use both the information from the feature description as well as the feature
identifier when possible, focusing on where there is reliable information. Report the
following results:
- feature identifier(s),
- feature(s) descriptions,
- bio_property which is the complete list of biological properties represented by 
these feature(s) (e.g. a pathway or biological complex), 
- env_property which is the complete list of environmental properties from this 
environment for which these feature(s) are an advantage (e.g., nutrient poor, high pH).
- estimated confidence value for this inferred association based on a scale of
unknown, low, medium, high
- very brief text explanation

Make sure that env_property does not include biological focused terms like 
'DNA damage repair' or 'cellular maintenance'. Instead focus on geochemical, 
geophysical, physical, chemical, and other abiological aspects of the environment.

The problem does not involve finding relationships of feature(s) to different ecosystems
or any ecosystem but rather to a very specific ecosystem, in this case defined as 
<Ecosystem> according to the GOLD environmental classification. Prioritize reporting 
relationships that involve groups of multiple features but still report results for each 
individual feature. 

It is critical that the generated results and inferences are thorough, 
exhaustive, and reliable.

Return a JSON structure of results like this and critically the lists of values need to use '*' as the list delimiter throughout:
{
    <Ecosystem> : {
        1 : {
            "feature_id": "IPR007210 * GO:0031460",
            "feature_label": "Glycine betaine/proline betaine transport system ATP-binding protein ProV-like * glycine betaine transport",
            "bio_property": "osmoprotection * stress response",
            "env_property": "marine * high salinity",
            "confidence": "high",
            "explanation" : "glycine betaine is a known osmoprotectant in high osmolarity environments"
        },
        2 : {
            ...
        },
        3 : ...
    }
}

Below is the specific list of biological features that were observed in the <Ecosystem> ecosystem:
<Features_and_Taxa>

***End Environmental Ecology and Microbiology Prompt
"""


In [188]:
prompt_template = """
***Begin Environmental Ecology and Microbiology Prompt***

You are an expert in microbiology, ecology, and environmental science. Your task is to 
identify and explain relationships between microbial metagenomic features (including some
eukaryotic features) and the environmental properties of a specific ecosystem.

**Context:**
In our experiment within the <Ecosystem> ecosystem, we observed various important features
such as Gene Ontology (GO) terms, InterPro (IPR) sequence domains, and NCBI Taxonomy (sk__).
These features are represented by standard identifiers and descriptions.

**Objective:**
For each feature or feature subset, identify and explain the relationships between these
features and specific environmental properties. Use the feature descriptions and 
identifiers to provide reliable information.

**Required Results:**
For each feature or subset, report:
- **Feature Identifier(s)**
- **Feature Description(s)**
- **Bio_property**: List all biological properties of the features (e.g., pathways, biological complexes) excluding generic biological terms or just reiterating what the specific feature is.
- **Env_property**: List all relevant environmental properties of the environment (e.g., nutrient poor,
high pH) excluding biological terms like 'DNA damage repair' or 'cellular maintenance' or kinds of metabolism or biological functions.
- **Confidence Level**: Estimate the confidence of the inferred association (unknown, low, medium, high).
- **Brief Explanation**: Provide a concise explanation of the relationship.

**JSON Output Structure:**
The results should be returned in the following JSON format, using '*' as the list delimiter:

```json
{
    "<Ecosystem>": {
        "1": {
            "feature_id": "IPR007210 * GO:0031460",
            "feature_label": "Glycine betaine/proline betaine transport system ATP-binding protein ProV-like * glycine betaine transport",
            "bio_property": "osmoprotection * stress response",
            "env_property": "marine * high salinity",
            "confidence": "high",
            "explanation": "Glycine betaine is a known osmoprotectant in high osmolarity environments."
        },
        "2": { ... },
        "3": { ... }
    }
}

Important Notes:
- Focus on relationships between the features themselves and the between features and specific propeties
of the <Ecosystem> defined by the GOLD environmental classification.
- Prioritize reporting relationships involving groups of multiple features, the more features are
discovered to share a biological property or relate to an environmental property or properties the better.
- Do not report unknown or low confidence results.
- Ensure the results are thorough, exhaustive, and reliable.


Observed Features in the <Ecosystem> ecosystem:
<Features_and_Taxa>

***End Environmental Ecology and Microbiology Prompt***
"""

In [189]:
%%time

llm_results = []
for index, row in eco_feat_df.iterrows():
    ecosystem = row['Ecosystem']
    features_and_taxa = row['Features_and_Taxa']
    features_list = ast.literal_eval(features_and_taxa)

    # Replace feature IDs with labels
    formatted_features_list = [parsed_features.get(feature, feature) for feature in features_list]
    formatted_features = "\n".join(formatted_features_list)

    # Generate prompt
    prompt = prompt_template.replace("<Ecosystem>", ecosystem.replace("root:", "")).replace("<Features_and_Taxa>", formatted_features)

    # Print the prompt (for debugging)
    print(prompt)

    # Query the LLM
    result = query_openai("gpt-4o", "user", prompt)
    llm_results.append(result)

    # Print the result (for debugging)
    print(f"Result for row {index}:\n{result}\n")


***Begin Environmental Ecology and Microbiology Prompt***

You are an expert in microbiology, ecology, and environmental science. Your task is to 
identify and explain relationships between microbial metagenomic features (including some
eukaryotic features) and the environmental properties of a specific ecosystem.

**Context:**
In our experiment within the Engineered:Bioreactor ecosystem, we observed various important features
such as Gene Ontology (GO) terms, InterPro (IPR) sequence domains, and NCBI Taxonomy (sk__).
These features are represented by standard identifiers and descriptions.

**Objective:**
For each feature or feature subset, identify and explain the relationships between these
features and specific environmental properties. Use the feature descriptions and 
identifiers to provide reliable information.

**Required Results:**
For each feature or subset, report:
- **Feature Identifier(s)**
- **Feature Description(s)**
- **Bio_property**: List all biological properties of 

In [190]:
import json
import re
import pandas as pd

def clean_json_content(content):
    # Remove markdown code block markers
    content = content.strip().replace('```json', '').replace('```', '')

    # Remove square brackets from values and convert lists to strings
    def replace_list(match):
        items = match.group(1).split(',')
        items = [item.strip().strip('"') for item in items]
        return '"' + ', '.join(items) + '"'
    
    content = re.sub(r'\[(.*?)\]', replace_list, content)
    
    # Ensure there are no double quotes around individual items
    content = re.sub(r'""', '"', content)
    
    return content

def parse_json_result(result):
    parsed_data = []
    try:
        result_dict = json.loads(result)
    except json.JSONDecodeError as e:
        print(f"JSON decode error for result: {e}")
        print("Content that caused the error:", result)
        return parsed_data

    for ecosystem, entries in result_dict.items():
        for entry_id, entry_data in entries.items():
            parsed_data.append({
                'ecosystem': ecosystem,
                'feature_id': entry_data.get('feature_id', ''),
                'feature_label': entry_data.get('feature_label', ''),
                'bio_property': entry_data.get('bio_property', '').replace(' * ', '*'),
                'env_property': entry_data.get('env_property', '').replace(' * ', '*'),
                'confidence': entry_data.get('confidence', ''),
                'explanation': entry_data.get('explanation', '')
            })
    return parsed_data

# Initialize an empty list to hold all parsed results
all_parsed_data = []

# Loop over the llm_results and parse each result
for result in llm_results:
    curdict = result.to_dict()
    content = curdict['choices'][0]['message']['content']
    
    try:
        # Clean the content to ensure proper JSON format
        content = clean_json_content(content)
        
        # Ensure the content starts and ends with braces
        content = content.strip()
        if content.startswith('{') and content[1] == '{':
            content = content[1:]
        if content.endswith('}') and content[-2] == '}':
            content = content[:-1]
        
        parsed_data = parse_json_result(content)
        print(parsed_data)
        all_parsed_data.extend(parsed_data)
    except Exception as e:
        print(f"Unexpected error for result: {e}")
        print("Content that caused the error:", content)

# Create a dataframe from the parsed data
llm_results_df = pd.DataFrame(all_parsed_data)

llm_results_df


JSON decode error for result: Expecting value: line 1 column 1 (char 0)
Content that caused the error: Sure. Below is the detailed relationships between observed features and environmental properties of the Engineered:Bioreactor ecosystem, represented in the desired JSON format:


{
    "Engineered:Bioreactor": {
        "1": {
            "feature_id": "GO:0018551 * IPR005126",
            "feature_label": "dissimilatory sulfite reductase activity * NapC/NirT cytochrome c, N-terminal",
            "bio_property": "sulfate reduction * anaerobic respiration",
            "env_property": "anaerobic * high sulfate",
            "confidence": "high",
            "explanation": "Dissimilatory sulfite reductase is essential for the reduction of sulfates in anaerobic conditions where high sulfate concentration is prevalent."
        },
        "2": {
            "feature_id": "IPR004763 * IPR016300",
            "feature_label": "Cation efflux system CzcA/CusA/SilA/NccA/HelA/CnrA * Arsenical 

,ecosystem,feature_id,feature_label,bio_property,env_property,confidence,explanation
0,Engineered:Bioremediation:Terephthalate:Wastew...,GO:0004114 * IPR004501,"3',5'-cyclic-nucleotide phosphodiesterase acti...",signal transduction*sugar transport,high organic load*nutrient dense,high,Systems involved in signal transduction and su...
1,Engineered:Bioremediation:Terephthalate:Wastew...,GO:0008901 * IPR010960,ferredoxin hydrogenase activity * Flavocytochr...,electron transport chain*anaerobic respiration,oxygen-limited*high organic load,high,Ferredoxin hydrogenase and Flavocytochrome c a...
2,Engineered:Bioremediation:Terephthalate:Wastew...,GO:0019512 * IPR004300,lactose catabolic process via tagatose-6-phosp...,lactose metabolism*carbohydrate degradation,high organic load*carbohydrate-rich conditions,high,These features are involved in carbohydrate me...
3,Engineered:Bioremediation:Terephthalate:Wastew...,GO:0004037 * IPR017813,allantoicase activity * Mycothiol acetyltransf...,amino acid metabolism*detoxification,nutrient dense*potentially toxin-rich,medium,Enzymes involved in amino acid metabolism and ...
4,Engineered:Bioremediation:Terephthalate:Wastew...,IPR014580 * IPR014984,Uncharacterised conserved protein UCP033199 * ...,unknown function*pathogenicity,diverse microbial community*complex interactions,medium,Presence of pathogenicity-related proteins ind...
...,...,...,...,...,...,...,...
131,Host-associated:Plants,IPR001096 * IPR033126,"Peptidase C13, legumain * Glycosyl hydrolases ...",protein degradation*carbohydrate degradation,nutrient cycling*decomposition,high,Peptidases and glycosyl hydrolases are essenti...
132,Host-associated:Plants:Phylloplane,GO:0042176 * IPR006652,regulation of protein catabolic process * Kelc...,protein degradation*structural protein interac...,nutrient limited*exposure to microbial competi...,high,The regulation of protein catabolic processes ...
133,Host-associated:Plants:Phylloplane,IPR000036 * GO:0033890,"Peptidase A26, omptin * ribonuclease D activity",protein degradation*RNA processing,high microbial diversity*exposure to pathogeni...,high,Peptidases and ribonucleases are necessary for...
134,Host-associated:Plants:Phylloplane,GO:0003950 * IPR033126,NAD+ ADP-ribosyltransferase activity * Glycosy...,DNA repair*carbohydrate metabolism,exposure to UV light*abundant plant polysaccha...,high,NAD+ ADP-ribosyltransferase activity is key fo...


In [191]:
llm_results_df

,ecosystem,feature_id,feature_label,bio_property,env_property,confidence,explanation
0,Engineered:Bioremediation:Terephthalate:Wastew...,GO:0004114 * IPR004501,"3',5'-cyclic-nucleotide phosphodiesterase acti...",signal transduction*sugar transport,high organic load*nutrient dense,high,Systems involved in signal transduction and su...
1,Engineered:Bioremediation:Terephthalate:Wastew...,GO:0008901 * IPR010960,ferredoxin hydrogenase activity * Flavocytochr...,electron transport chain*anaerobic respiration,oxygen-limited*high organic load,high,Ferredoxin hydrogenase and Flavocytochrome c a...
2,Engineered:Bioremediation:Terephthalate:Wastew...,GO:0019512 * IPR004300,lactose catabolic process via tagatose-6-phosp...,lactose metabolism*carbohydrate degradation,high organic load*carbohydrate-rich conditions,high,These features are involved in carbohydrate me...
3,Engineered:Bioremediation:Terephthalate:Wastew...,GO:0004037 * IPR017813,allantoicase activity * Mycothiol acetyltransf...,amino acid metabolism*detoxification,nutrient dense*potentially toxin-rich,medium,Enzymes involved in amino acid metabolism and ...
4,Engineered:Bioremediation:Terephthalate:Wastew...,IPR014580 * IPR014984,Uncharacterised conserved protein UCP033199 * ...,unknown function*pathogenicity,diverse microbial community*complex interactions,medium,Presence of pathogenicity-related proteins ind...
...,...,...,...,...,...,...,...
131,Host-associated:Plants,IPR001096 * IPR033126,"Peptidase C13, legumain * Glycosyl hydrolases ...",protein degradation*carbohydrate degradation,nutrient cycling*decomposition,high,Peptidases and glycosyl hydrolases are essenti...
132,Host-associated:Plants:Phylloplane,GO:0042176 * IPR006652,regulation of protein catabolic process * Kelc...,protein degradation*structural protein interac...,nutrient limited*exposure to microbial competi...,high,The regulation of protein catabolic processes ...
133,Host-associated:Plants:Phylloplane,IPR000036 * GO:0033890,"Peptidase A26, omptin * ribonuclease D activity",protein degradation*RNA processing,high microbial diversity*exposure to pathogeni...,high,Peptidases and ribonucleases are necessary for...
134,Host-associated:Plants:Phylloplane,GO:0003950 * IPR033126,NAD+ ADP-ribosyltransferase activity * Glycosy...,DNA repair*carbohydrate metabolism,exposure to UV light*abundant plant polysaccha...,high,NAD+ ADP-ribosyltransferase activity is key fo...


In [192]:
llm_results_df[llm_results_df["confidence"] == "high"]

,ecosystem,feature_id,feature_label,bio_property,env_property,confidence,explanation
0,Engineered:Bioremediation:Terephthalate:Wastew...,GO:0004114 * IPR004501,"3',5'-cyclic-nucleotide phosphodiesterase acti...",signal transduction*sugar transport,high organic load*nutrient dense,high,Systems involved in signal transduction and su...
1,Engineered:Bioremediation:Terephthalate:Wastew...,GO:0008901 * IPR010960,ferredoxin hydrogenase activity * Flavocytochr...,electron transport chain*anaerobic respiration,oxygen-limited*high organic load,high,Ferredoxin hydrogenase and Flavocytochrome c a...
2,Engineered:Bioremediation:Terephthalate:Wastew...,GO:0019512 * IPR004300,lactose catabolic process via tagatose-6-phosp...,lactose metabolism*carbohydrate degradation,high organic load*carbohydrate-rich conditions,high,These features are involved in carbohydrate me...
6,Engineered:Bioremediation:Terephthalate:Wastew...,sk__Bacteria;k__;p__Bacteroidetes;c__Bacteroid...,Bacteroides * Collinsella,complex carbohydrate breakdown*secondary metab...,high fiber*complex organic matter,high,Bacteroides and Collinsella are known for thei...
7,Engineered:Built environment,IPR009413,"Hemolysin, aegerolysin type",cell lysis*pathogenicity,humid*high dust load,high,Hemolysins like aegerolysin type are often ass...
...,...,...,...,...,...,...,...
131,Host-associated:Plants,IPR001096 * IPR033126,"Peptidase C13, legumain * Glycosyl hydrolases ...",protein degradation*carbohydrate degradation,nutrient cycling*decomposition,high,Peptidases and glycosyl hydrolases are essenti...
132,Host-associated:Plants:Phylloplane,GO:0042176 * IPR006652,regulation of protein catabolic process * Kelc...,protein degradation*structural protein interac...,nutrient limited*exposure to microbial competi...,high,The regulation of protein catabolic processes ...
133,Host-associated:Plants:Phylloplane,IPR000036 * GO:0033890,"Peptidase A26, omptin * ribonuclease D activity",protein degradation*RNA processing,high microbial diversity*exposure to pathogeni...,high,Peptidases and ribonucleases are necessary for...
134,Host-associated:Plants:Phylloplane,GO:0003950 * IPR033126,NAD+ ADP-ribosyltransferase activity * Glycosy...,DNA repair*carbohydrate metabolism,exposure to UV light*abundant plant polysaccha...,high,NAD+ ADP-ribosyltransferase activity is key fo...


In [193]:
llm_results_df.to_csv("ecosystem_characteristics_llm_results_df_v5.tsv",sep="\t",header=True)